In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd
from source_code.backpip import data_up_pip

Autosave disabled
Local Sec Env loaded


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
328,AFG,Asia,Afghanistan,2021-01-17,53984.0,46.0,70.714,2339.0,3.0,8.857,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
643,ALB,Europe,Albania,2021-01-17,67690.0,474.0,585.000,1277.0,7.0,5.143,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
971,DZA,Africa,Algeria,2021-01-17,103833.0,222.0,241.286,2836.0,5.0,4.143,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## insert yesterday reporting to bdd
# try:
#     verif_data = verif_data.replace({np.nan: 'Null'})
#     data_job = data_up_pip(verif_data) 
#     data_job.up_baseline()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")

In [6]:
%%time
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=2)
yesterday_date = str(yesterday_date)
this_date = date.today() - timedelta(days=1)
this_date = str(this_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

Wall time: 1.81 s


In [9]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    df = df[df['date'].isin([f"{this_date}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]

    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2
    
    ind = 0    
    follow_df = pd.DataFrame()
    follow_df.at[ind,"date"] = v0
    follow_df.at[ind,"country"] = country
    follow_df.at[ind,"total_cases_predict"] = rez1
    follow_df.at[ind,"total_cases_real"] = v1
    follow_df.at[ind,"total_deaths_predict"] = rez2
    follow_df.at[ind,"total_deaths_real"] = v2
    follow_df.at[ind,"error_abs_cases"] = errorCase1
    follow_df.at[ind,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [10]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
# follow_df4 = Eval(c,data_load,"Belgium")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [11]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,error_abs_cases,error_abs_deaths
0,2021-01-17,France,-8741.0,218.0
0,2021-01-17,China,-76.0,2.0
0,2021-01-17,Italy,8493.0,87.0
0,2021-01-17,United States,178932.0,-151.0
0,2021-01-17,World,573471.0,1115.0
0,2021-01-17,United Kingdom,26460.0,308.0
0,2021-01-17,Germany,15021.0,222.0
0,2021-01-17,Iran,8299.0,470.0
0,2021-01-17,Turkey,19289.0,89.0
0,2021-01-17,Brazil,1521.0,164.0


In [12]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [9]:
# # insert yesterday reporting to bdd
# try:
#     data_job = data_up_pip(rapport) 
#     data_job.up_rapp()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")

 Insert Rapport Done !! 
